### Exercise 1. (EM for Gaussian random walk)

1. Implement the $EM$ algorithm for estimation of the measurement noise variance in the Gaussian random walk model. 
2. Simulate data with 256 time steps and test the algorithm. How many iterations are needed for convergence?

### Solution Exercise 1.

Recall that the random walk can be written in state space form in the following way:

$$
x_k = x_{k - 1} + q  \qquad q \sim \mathcal{N}(0, Q)
$$
$$
y_k = x_{k} + r \qquad r \sim \mathcal{N}(0, R)
$$

In order to derive the $EM$ algorithm for the random walk we should write down the complete data log-likelihood for the model given a set of parameters $\mathbf{\theta}$ which is:

$$
 \log p(\mathbf{y}_{1:T}, \mathbf{x}_{0:T} | \theta) = \log(x_0|\theta) + \sum_{k=1}^T \log p(x_k|x_{k-1}, \theta) + \sum_{k=1}^T \log p(y_k | x_k,\theta)
$$

To make the $E$ step we define the function $Q(\theta, \theta^{(n)})$:

$$
\begin{align}
Q(\theta, \theta^{(n)})& = E[\log p(\mathbf{y}_{1:k}, \mathbf{x}_{0:k} | \theta)] \\
                       & = E[\log(x_0|\theta)] + \sum_{k=1}^T E[ \log p(x_k|x_{k-1}, \theta) ] + \sum_{k=1}^T E[\log p(y_k | x_k,\theta)]
\end{align}
$$

where the expectation $E[.]$ is defined in terms of $p(\mathbf{x}_{0:T} | \theta^{(n)}, \mathbf{y}_{1:T})$. Formally,

$$
E[g(x)] = \int g(\mathbf{x}_{0:T}) p(\mathbf{x}_{0:T} | \theta^{(n)}, \mathbf{y}_{1:T}) d\mathbf{x}_{0:T}
$$

Recall that in this case the parameter we want to estimate is the measurement noise variance ($R$) and it only affects $ p(y_k | x_k)$. Then the function $Q(.,.)$ can be rewritten as follows:


$$
\begin{align}
Q(R, R^{(n)})& =  \sum_{k=1}^T E[\log p(y_k | x_k, R)] + constant \\
                       & = \sum_{k=1}^T \int \log p(y_k | x_k, R) p(x_k | R^{(n)}, \mathbf{y}_{1:T}) d{x}_{k} + constant
\end{align}
$$

Now we need to derive the $M$ step. In this step we have to optimize $Q(.,.)$ with respect to $R$. Expanding the last expression we get:



$$
\begin{align}
Q(R, R^{(n)}) & = \sum_{k=1}^T \int \bigg(-\frac{1}{2}\log 2\pi - \frac{1}{2}\log R - \frac{(y_k - x_k)^2}{2R}\bigg) \mathcal{N}(m_k^s, P_k^s) d{x}_{k} + constant
\end{align}
$$